# CNN model with custom dataset

- Data preprocessing
- Defining the CNN class
- Training and testing the model
- Make inference

In [39]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam, lr_scheduler
from torch.autograd import Variable
import torchvision
import pathlib
import glob
import random
import shutil
 

In [40]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### Splitting the dataset into training and testing sets

In [41]:
# Extracting the dataset and creating the train and test folders
data_dir = pathlib.Path('30by30 images')
classes = os.listdir(data_dir)
print('classes: ',classes)

# Creating the train and test folders
if not os.path.exists('./custom_cnn_dataset'):
    os.makedirs('./custom_cnn_dataset/train')
    os.makedirs('./custom_cnn_dataset/test')

    # Creating the class folders in train and test folders
    for i in classes:
        os.makedirs('./custom_cnn_dataset/train/' + i)
        os.makedirs('./custom_cnn_dataset/test/' + i)

random.seed(0)
# Splitting the dataset into train and test sets
for i in classes:
    src = "./30by30 images/" + i # Folder to copy images from
    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)*0.8)])
    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]
    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Testing: ', len(test_FileNames))
    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, "./custom_cnn_dataset/train/" + i)
    for name in test_FileNames:
        shutil.copy(name, "./custom_cnn_dataset/test/" + i)

classes:  ['1', '2', '3']
Total images:  596
Training:  476
Testing:  120
Total images:  588
Training:  470
Testing:  118
Total images:  626
Training:  500
Testing:  126


#### Data preprocessing

In [42]:
transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [43]:
train_dir = './custom_cnn_dataset/train'
train_dataloader = DataLoader(
    torchvision.datasets.ImageFolder(train_dir, transform=transforms),
    batch_size=64, shuffle=True
)

test_dir = './custom_cnn_dataset/test'
test_dataloader = DataLoader(
    torchvision.datasets.ImageFolder(test_dir, transform=transforms),
    batch_size=64, shuffle=True
)

#### CNN class defination

In [44]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ConvNet, self).__init__()
        
        # Input shape = (64, 3, 30, 30)
        # Output size after convolutional layer = (w-f+2p)/s + 1 = (30-3+2)/1 + 1 = 30
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        #Input shape = (64, 12, 30, 30)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        
        # Input shape = (64, 12, 30, 30)
        # Output size after max pooling = 30/2 = 15
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Input shape = (64, 12, 15, 15)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()
        #shape = (64, 32, 15, 15)
        
        self.fc = nn.Linear(in_features=15*15*32, out_features=num_classes)
    
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        
        output = self.maxpool1(output)
        
        output = self.conv2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        
        #reshaping the output to feed into the fully connected layer
        output = output.view(-1, 15*15*32)
        
        output = self.fc(output)
        
        return output

In [45]:
model = ConvNet(num_classes=3).to(device)

In [46]:
# Loss and optimizer
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [47]:
#train and test count
train_count = len(glob.glob(train_dir + '/**/*.png'))
test_count = len(glob.glob(test_dir + '/**/*.png'))
print('train_count: ', train_count)
print('test_count: ', test_count)

train_count:  1808
test_count:  1219


In [48]:
#training the model and saving best model
num_epochs = 10
best_accuracy = 0.0

for epoch in range(num_epochs):
    #Evaluation and training on training dataset
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i, (images, labels) in enumerate(train_dataloader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data*images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        
        train_accuracy += int(torch.sum(prediction==labels.data))
        
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    #Evaluation on testing dataset
    model.eval()
    test_accuracy = 0.0
    for i, (images, labels) in enumerate(test_dataloader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction==labels.data))
        
    test_accuracy = test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy
        
    scheduler.step()

Epoch: 0 Train Loss: tensor(0.6873) Train Accuracy: 0.7157079646017699 Test Accuracy: 0.8777686628383922
Epoch: 1 Train Loss: tensor(0.2042) Train Accuracy: 0.9391592920353983 Test Accuracy: 0.9327317473338802
Epoch: 2 Train Loss: tensor(0.1085) Train Accuracy: 0.9662610619469026 Test Accuracy: 0.9196062346185397
Epoch: 3 Train Loss: tensor(0.0769) Train Accuracy: 0.9712389380530974 Test Accuracy: 0.9647251845775225
Epoch: 4 Train Loss: tensor(0.0495) Train Accuracy: 0.9823008849557522 Test Accuracy: 0.9721082854799016
Epoch: 5 Train Loss: tensor(0.0283) Train Accuracy: 0.9928097345132744 Test Accuracy: 0.9958982772764561
Epoch: 6 Train Loss: tensor(0.0256) Train Accuracy: 0.9939159292035398 Test Accuracy: 0.9983593109105825
Epoch: 7 Train Loss: tensor(0.0236) Train Accuracy: 0.9972345132743363 Test Accuracy: 0.9983593109105825
Epoch: 8 Train Loss: tensor(0.0195) Train Accuracy: 0.9983407079646017 Test Accuracy: 0.9983593109105825
Epoch: 9 Train Loss: tensor(0.0191) Train Accuracy: 0.9